# Spider启动详解
　　参考：[scrapy启动流程图（超详细）——cmdline.py解析](https://blog.csdn.net/bf96163/article/details/112834661)<br>
　　　　　[Scrapy crawler.py 代码详细解析](https://blog.csdn.net/bf96163/article/details/112536234)<br>
　　　　　[Scrapy源码剖析（二）Scrapy是如何运行起来的？](https://zhuanlan.zhihu.com/p/272370864)<br>
　　　　　[scrapy 源码解析 （一）：启动流程源码分析(一)命令行启动](https://www.cnblogs.com/qiu-hua/p/12930422.html)<br>
　　　　　[Scrapy源码阅读分析_3_核心组件](https://blog.csdn.net/freeking101/article/details/87639667)<br>
　　　　　[Scrapy 源码阅读（二）：看源码](https://zhuanlan.zhihu.com/p/150120517)<br>
　　　　　[Scrapy源码阅读分析_2_启动流程](https://blog.csdn.net/freeking101/article/details/87639139)<br>
　　　　　[Scrapy源码阅读分析_4_请求处理流程](https://blog.csdn.net/freeking101/article/details/87707531)<br>

## 一、简介
　　我们知道命令 scrapy crawler spidername 可以启动并运行爬虫程序，那么这个命令背后都做了些什么，Spider是如何被启动的呢？本单元就来研究这个内容。<br>
　　总体来讲，scrapy爬虫启动做了两件事：<br>
　　1、导入项目，并根据settings初始化项目。<br>
　　2、启动一个CrawlerProcess进程，用于维护爬虫的运行，并获取reactor事件循环，用于等待执行各个爬虫的异步任务。<br>
## 二、命令入口点
### 2.1、命令里的scrapy是一个可执行文件，后面的crawl myspider是scrapy的参数。<BR>
　　可执行文件scrapy在/usr/local/python/bin目录里（linux系统使用which scrapy查看），是一个python脚本，有效代码为：<BR>
>from scrapy.cmdline import execute<BR>
<BR>
if __name__ == '__main__':<BR>
　　sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv\[0])<BR>
　　sys.exit(execute())<BR>

　　这个文件的作用就是从命令行里读取命令，然后传递给scrapy.cmdline包的execute()方法进行下一步操作（scrapy项目启动入口点）。<BR>
### 2.2 scrapy命令的生成原理
　　安装scrapy后, scrapy的setup.py文件生明并指定了程序的入口点为：scrapy.cmdline.execute方法。  
　　首先我们查看一下setup.py：<br>
>entry_points={<br>
　　'console_scripts': \['scrapy = scrapy.cmdline:execute'\]<br>
    },<br>

　　可以看到，框架唯一的入口点是命令行的scrapy命令，对应scrapy.cmdline下的execute方法。<br>
　　也就是说，我们在安装 Scrapy 的过程中，setuptools 这个包管理工具，就会把上述代码生成好并放在可执行路径下，这样当我们调用 scrapy 命令时，就会调用 Scrapy 模块下的 cmdline.py 的 execute 方法。<br>
## 三、cmdline.py与execute方法
　　![cmdline.py的代码逻辑](./images/scrapy_cmdline爬虫启动逻辑.png)<br>
　　cmdline.py的excute方法，可以看做整个scrapy项目的起点，其整体操作围绕着两个部分，一个是命令解析与配置初始化;一个是crawler/crawprocess爬虫类加载与运行。<br>


     
     

In [1]:
# --coding:utf-8--
# scrapy.cmdline.py
from __future__ import print_function
import sys
import os
import optparse
import cProfile
import inspect
import pkg_resources
 
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.commands import ScrapyCommand
from scrapy.exceptions import UsageError
from scrapy.utils.misc import walk_modules
from scrapy.utils.project import inside_project, get_project_settings
from scrapy.utils.python import garbage_collect
 
 
def _iter_command_classes(module_name):
    # TODO: add `name` attribute to commands and and merge this function with
    # scrapy.utils.spider.iter_spider_classes
    # XX实现从模块名字到模块的映射XX
    """ 递归迭代module_name下的所有模块和子模块,并判断该模块是否是 Scrapycommand 的子类，是的话就
    抛出这个子模块；其中walk_modules 是将一个模块内的所有可用模块列出，包括子模块；
    vars（obj）函数返回的是一个包含obj的所有属性与其值的字典对象"""
    for module in walk_modules(module_name):
        for obj in vars(module).values():
            if (inspect.isclass(obj)   # 是类
                and issubclass(obj, ScrapyCommand)    # 是Scrapy命令基类的子类
                and obj.__module__ == module.__name__    # 是module下面的类 而不是其他模块下的类
                and not obj == ScrapyCommand       # 是继承后的而不是本体
            ):
                yield obj
 

def _get_commands_from_module(module, inproject):
    """给定一个模块 和是否在项目内的flag（inproject，用于区别全局命令和项目命令），检查
    _iter_command_classes返回的 Scrapycommand 的子类，并按inproject筛选符合条件的命令集的实列返回
    """
    d = {}
    for cmd in _iter_command_classes(module):
        # 在项目内时候，且目标函数不需要在项目内的时候 添加到函数字典中
        if inproject or not cmd.requires_project:     # ScrapyCommand.requires_project 默认为false
            cmdname = cmd.__module__.split('.')[-1]    # 命令名称字符串
            d[cmdname] = cmd(）              # 构造符合条件的命令的对象实列映射集
    return d
 

def _get_commands_from_entry_points(inproject, group='scrapy.commands'):
    """本函数用于扩展自定义的scrapy命令的引入。方法意义在于项目set_up文件定义entry_point时，
    可以导入自定义的命令；或是encry_point.txt配置文件
    """
    cmds = {}
    for entry_point in pkg_resources.iter_entry_points(group):
        obj = entry_point.load()    # 获取指定包下的对象
        if inspect.isclass(obj):
            cmds[entry_point.name] = obj()   # 命令名称与命令对象实例
        else:
            raise Exception(f"Invalid entry point {entry_point.name}")
    return cmds
 
 
def _get_commands_dict(settings, inproject):
    """ 从上面的两个方法中拿到所有的模块，同时如果setting中有 'COMMANDS_MODULE' 再把这些模块键入到模块列表中"""
    cmds = _get_commands_from_module('scrapy.commands', inproject)   # 返回scrapy框架本身定义的命令与实列的映射集
    # 寻求其他group拓展入口点的命令映射集，本处使用默认scrapy.commands,既无自定义命令，前后cmds不变
    cmds.update(_get_commands_from_entry_points(inproject))  
    cmds_module = settings['COMMANDS_MODULE']     # 在settings.py中指明的自定义commands，加入到cmds
    if cmds_module:     # COMMANDS_MODULE默认''
        cmds.update(_get_commands_from_module(cmds_module, inproject))
    return cmds
 
 
def _pop_command_name(argv):
    # 功能是从命令行参数中找到第一个不以 - 开头的命令，返回这个命令：
    i = 0
    for arg in argv[1:]:
        if not arg.startswith('-'):
            del argv[i]
            return arg
        i += 1
 
 
def _print_header(settings, inproject):
    version = scrapy.__version__
    if inproject:
        print(f"Scrapy {version} - project: {settings['BOT_NAME']}\n")
    else:
        print(f"Scrapy {version} - no active project\n")
 

def _print_commands(settings, inproject):
    #将对应所有的方法打印出来
    _print_header(settings, inproject)
    print("Usage:")
    print("  scrapy <command> [options] [args]\n")
    print("Available commands:")
    cmds = _get_commands_dict(settings, inproject)
    for cmdname, cmdclass in sorted(cmds.items()):
        print(f"  {cmdname:<13} {cmdclass.short_desc()}")
    if not inproject:
        print()
        print("  [ more ]      More commands available when run from project directory")
    print()
    print('Use "scrapy <command> -h" to see more info about a command')
 
 
def _print_unknown_command(settings, cmdname, inproject):
    _print_header(settings, inproject)
    print(f"Unknown command: {cmdname}\n")
    print('Use "scrapy" to see available commands')
 

def _run_print_help(parser, func, *a, **kw):
    # 包装了一下输入函数，当func报错时输出help信息。
    #调用函数 出错的话 用parser 传递相应问题
    try:
        func(*a, **kw)
    except UsageError as e:
        if str(e):
            parser.error(str(e))
        if e.print_help:
            parser.print_help()
        sys.exit(2)


def execute(argv=None, settings=None):
    # 首先是配置了argv和settings变量
    # 是否用其他方式传入命令参数，否的话使用命令行参数
    if argv is None:
        argv = sys.argv
 
    if settings is None:
        # 没指定setting的话，就调用默认方法
        # 从全局配置文件scrapy.cfg载入settings，再从环境中载入scrapy相关的settings到setting对象里
        settings = get_project_settings()  
        # set EDITOR from environment if available 用于编辑文件
        try:
            editor = os.environ['EDITOR']
        except KeyError:
            pass
        else:
            settings['EDITOR'] = editor
    check_deprecated_settings(settings)   # 检查已弃用的设置，如发现，输出一行警告信息。
 
    inproject = inside_project()  # 判断执行环境是否在项目中 主要检查scrapy.cfg配置文件是否存在
    cmds = _get_commands_dict(settings, inproject)     # 拿到当前项目状态下所有可用命令集
    cmdname = _pop_command_name(argv)          # 获取启动输入的命令
    parser = optparse.OptionParser(formatter=optparse.TitledHelpFormatter(),
                                   conflict_handler='resolve')   # 指定一个 optparse 解析器
    if not cmdname:      # 未解析出命令行输入命令
        _print_commands(settings, inproject)
        sys.exit(0)
    elif cmdname not in cmds: # 解析出的命令字符不在可用命令集内
        _print_unknown_command(settings, cmdname, inproject)
        sys.exit(2)
 
    cmd = cmds[cmdname]    # 拿到输入命令的实列
    # 使用cmd的属性（usage和description）对于parser的属性做了设定。
    # 然后将settings赋值给了cmd.settings，将parser赋给了cmd.add_options。
    parser.usage = f"scrapy {cmdname} {cmd.syntax()}"
    parser.description = cmd.long_desc()
    settings.setdict(cmd.default_settings, priority='command')   # 将命令中的设置弄到settings中
    cmd.settings = settings
    cmd.add_options(parser)
    opts, args = parser.parse_args(args=argv[1:])
 
    # 运行命令
    _run_print_help(parser, cmd.process_options, args, opts)   # 主要是写一些setting
    # 生成CrawlerProcess实例，并给命令实例添加crawler_process属性
    cmd.crawler_process = CrawlerProcess(settings)
    _run_print_help(parser, _run_command, cmd, args, opts) # 调用cmd的run方法启动采集任务
    sys.exit(cmd.exitcode)
 
 
def _run_command(cmd, args, opts):
    if opts.profile:
        _run_command_profiled(cmd, args, opts)
    else:
        cmd.run(args, opts)      # 调用cmd的run方法启动采集任务
 

def _run_command_profiled(cmd, args, opts):
    # 是对cProfile做了包装，当log比较多的时候，使用cProfile可以比较方便的整理和查看。
    # 用cpython的porfiler 运行 cmd.run(args, opts) 并传入变量
    if opts.profile:
        sys.stderr.write(f"scrapy: writing cProfile stats to {opts.profile}\n")
    loc = locals()           # 拿到所有变量
    p = cProfile.Profile()
    p.runctx('cmd.run(args, opts)', globals(), loc)     # 调用函数 并传入 global 和 locals 中的变量
    if opts.profile:
        p.dump_stats(opts.profile)
 
 
if __name__ == '__main__':
    try:
        execute()
    finally:
        # Twisted prints errors in DebugInfo.__del__, but PyPy does not run gc.collect() on exit:
        # http://doc.pypy.org/en/latest/cpython_differences.html
        # ?highlight=gc.collect#differences-related-to-garbage-collection-strategies
        garbage_collect()

SyntaxError: invalid character in identifier (<ipython-input-1-1291515c5548>, line 46)

## 四、project.py与项目初始化配置
　　首先第一步，根据环境初始化配置。这主要和环境变量和 scrapy.cfg 有关，通过调用 get_project_settings 方法，最终生成一个 Settings 实例。在初始配置时，会加载默认的配置文件 default_settings.py，主要逻辑在 Settings 类中。<br>
　　检查运行环境是否在项目中<br>
　　初始化完配置之后，下面一步是检查运行环境是否在爬虫项目中。我们知道，scrapy 命令有的是依赖项目运行的，有的命令则是全局的。这里主要通过就近查找 scrapy.cfg 文件来确定是否在项目环境中，主要逻辑在 inside_project 方法中,就是判断能否找到 scrapy.cfg 文件，如果能找到，则说明是在爬虫项目中，否则就认为是执行的全局命令。<br>






In [ ]:
# --coding:utf-8--
# scrapy.utils.project.py
import os
from six.moves import cPickle as pickle
import warnings

from importlib import import_module
from os.path import join, dirname, abspath, isabs, exists

from scrapy.utils.conf import closest_scrapy_cfg, get_config, init_env
from scrapy.settings import Settings
from scrapy.exceptions import NotConfigured
from scrapy.exceptions import ScrapyDeprecationWarning

ENVVAR = 'SCRAPY_SETTINGS_MODULE'
DATADIR_CFG_SECTION = 'datadir'


def inside_project():
    # 试着导入了一下setting文件，如果导入成功，说明当前目录是工程文件的根目录，返回True。
    # 如果失败了，则需要递归的检查一下是否是在工程的子目录中，如是，返回True，不在工程目录中，返回False。
    scrapy_module = os.environ.get('SCRAPY_SETTINGS_MODULE')
    if scrapy_module is not None:
        try:
            import_module(scrapy_module)
        except ImportError as exc:
            warnings.warn("Cannot import scrapy settings module %s: %s" % (scrapy_module, exc))
        else:
            return True
    # 如果环境变量没有 就近查找scrapy.cfg 找得到就认为是在项目环境中
    return bool(closest_scrapy_cfg())


def project_data_dir(project='default'):
    """Return the current project data dir, creating it if it doesn't exist"""
    if not inside_project():
        raise NotConfigured("Not inside a project")
    cfg = get_config()
    if cfg.has_option(DATADIR_CFG_SECTION, project):
        d = cfg.get(DATADIR_CFG_SECTION, project)
    else:
        scrapy_cfg = closest_scrapy_cfg()
        if not scrapy_cfg:
            raise NotConfigured("Unable to find scrapy.cfg file to infer project data dir")
        d = abspath(join(dirname(scrapy_cfg), '.scrapy'))
    if not exists(d):
        os.makedirs(d)
    return d


def data_path(path, createdir=False):
    """
    Return the given path joined with the .scrapy data directory.
    If given an absolute path, return it unmodified.
    """
    if not isabs(path):
        if inside_project():
            path = join(project_data_dir(), path)
        else:
            path = join('.scrapy', path)
    if createdir and not exists(path):
        os.makedirs(path)
    return path


def get_project_settings():
    """创建一个Setting对象，使用'SCRAPY_SETTINGS_MODULE'初始化该对象，再返回该对象。"""
    # 首先判断是否设置了SCRAPY_SETTINGS_MODULE环境变量，这个环境变量用来指定工程的配置模块；
    # SCRAPY_SETTINGS_MODULE环境变量不存在时,将初始化环境变量init_env(project)
    if ENVVAR not in os.environ:   
        # 初始化环境:找到用户配置文件settings.py的SCRAPY_PROJECT，设置到环境变量SCRAPY_SETTINGS_MODULE中
        project = os.environ.get('SCRAPY_PROJECT', 'default')    # 默认为default
        init_env(project)  

    settings = Settings()  # 加载默认配置文件default_settings.py 生成settings实例
    settings_module_path = os.environ.get(ENVVAR)  # 取得用户配置文件
    if settings_module_path:    # 如果有用户配置 则覆盖默认配置
        settings.setmodule(settings_module_path, priority='project')

    # XXX: remove this hack  # 如果环境变量中有其他scrapy相关配置也覆盖
    pickled_settings = os.environ.get("SCRAPY_PICKLED_SETTINGS_TO_OVERRIDE")
    if pickled_settings:
        warnings.warn("Use of environment variable "
                      "'SCRAPY_PICKLED_SETTINGS_TO_OVERRIDE' "
                      "is deprecated.", ScrapyDeprecationWarning)
        settings.setdict(pickle.loads(pickled_settings), priority='project')

    # XXX: deprecate and remove this functionality
    env_overrides = {k[7:]: v for k, v in os.environ.items() if
                     k.startswith('SCRAPY_')}
    if env_overrides:
        settings.setdict(env_overrides, priority='project')

    return settings


## 五、conf.py
　　init_env首先调用get_config()获取cfg配置文件，这个配置文件获取的优先级是：<BR>
　　1、/etc/scrapy.cfg，c:\scrapy\scrapy.cfg<BR>
　　2、XDG_CONFIG_HOME环境变量指定的目录下的scrapy.cfg<BR>
　　3、~/.scrapy.cfg<BR>
　　4、当前执行目录下的scrapy.cfg或者父目录中的scrapy.cfg<BR>
　　由于1，2，3默认都不设置，所以就使用当前执行命令下的scrapy.cfg，也就是工程目录下的scrapy.cfg。<BR>
　　根据default = projectName.settings找到对应的配置模块，后面会执行一系列导入settings.py配置项的操作。<BR>

In [ ]:
# --coding:utf-8--
# scrapy.utils.conf.py
import os
import sys
import numbers
from operator import itemgetter

import six
if six.PY2:
    from ConfigParser import SafeConfigParser as ConfigParser
else:
    from configparser import ConfigParser

from scrapy.settings import BaseSettings
from scrapy.utils.deprecate import update_classpath
from scrapy.utils.python import without_none_values


def build_component_list(compdict, custom=None, convert=update_classpath):
    """Compose a component list from a { class: order } dictionary."""

    def _check_components(complist):
        if len({convert(c) for c in complist}) != len(complist):
            raise ValueError('Some paths in {!r} convert to the same object, '
                             'please update your settings'.format(complist))

    def _map_keys(compdict):
        if isinstance(compdict, BaseSettings):
            compbs = BaseSettings()
            for k, v in six.iteritems(compdict):
                prio = compdict.getpriority(k)
                if compbs.getpriority(convert(k)) == prio:
                    raise ValueError('Some paths in {!r} convert to the same '
                                     'object, please update your settings'
                                     ''.format(list(compdict.keys())))
                else:
                    compbs.set(convert(k), v, priority=prio)
            return compbs
        else:
            _check_components(compdict)
            return {convert(k): v for k, v in six.iteritems(compdict)}

    def _validate_values(compdict):
        """Fail if a value in the components dict is not a real number or None."""
        for name, value in six.iteritems(compdict):
            if value is not None and not isinstance(value, numbers.Real):
                raise ValueError('Invalid value {} for component {}, please provide ' \
                                 'a real number or None instead'.format(value, name))

    # BEGIN Backward compatibility for old (base, custom) call signature
    if isinstance(custom, (list, tuple)):
        _check_components(custom)
        return type(custom)(convert(c) for c in custom)

    if custom is not None:
        compdict.update(custom)
    # END Backward compatibility

    _validate_values(compdict)
    compdict = without_none_values(_map_keys(compdict))
    return [k for k, v in sorted(six.iteritems(compdict), key=itemgetter(1))]


def arglist_to_dict(arglist):
    """Convert a list of arguments like ['arg1=val1', 'arg2=val2', ...] to a
    dict
    """
    return dict(x.split('=', 1) for x in arglist)


def closest_scrapy_cfg(path='.', prevpath=None):
    """Return the path to the closest scrapy.cfg file by traversing the current
    directory and its parents
    在当前文件夹下，寻找有没有scrapy.cfg文件，如果没有找到，则递归遍历父文件夹，直到遍历到根目录。
    如果找到了scrapy.cfg文件，则返回scrapy.cfg的绝对路径。搜索最靠近当前当前路径的scrapy.cfg配置文件并返回其路径。
    """
    if path == prevpath:
        return ''
    path = os.path.abspath(path)
    cfgfile = os.path.join(path, 'scrapy.cfg')
    if os.path.exists(cfgfile):
        return cfgfile
    return closest_scrapy_cfg(os.path.dirname(path), path)


def init_env(project='default', set_syspath=True):
    """Initialize environment to use command-line tool from inside a project
    dir. This sets the Scrapy settings module and modifies the Python path to
    be able to locate the project module.
    首先把'SCRAPY_SETTINGS_MODULE'设置为.cfg下指明的settings文件，然后调用
    closest_scrapy_cfg()函数，将工程目录添加到系统的path当中。
    """
    cfg = get_config()
    # 获取到配置文件后设置系统环境变量SCRAPY_SETTINGS_MODULE为配置模块路径，
    if cfg.has_option('settings', project):
        # 将项目配置模块路径设置进系统环境变量
        os.environ['SCRAPY_SETTINGS_MODULE'] = cfg.get('settings', project)
    closest = closest_scrapy_cfg()
    # 将最近的scrapy.cfg模块路径放入系统路径使Python能够找到该模块导入
    if closest:
        projdir = os.path.dirname(closest)
        if set_syspath and projdir not in sys.path:
            sys.path.append(projdir)


def get_config(use_closest=True):
    """Get Scrapy config file as a ConfigParser
    返回一个SafeConfigParser()示例，用于对.cfg文件的包装。
    """
    sources = get_sources(use_closest)
    cfg = ConfigParser()
    cfg.read(sources)
    return cfg


def get_sources(use_closest=True):
    # 返回了一系列.cfg文件可能存在的路径，其中包括当前文件夹使用closest_scrapy_cfg()函数获取到的路径。
    xdg_config_home = os.environ.get('XDG_CONFIG_HOME') or \
        os.path.expanduser('~/.config')
    sources = ['/etc/scrapy.cfg', r'c:\scrapy\scrapy.cfg',
               xdg_config_home + '/scrapy.cfg',
               os.path.expanduser('~/.scrapy.cfg')]
    if use_closest:
        sources.append(closest_scrapy_cfg())
    return sources



## 六、命令执行
　　这里以crawl命令为例！<br>
　　scrapy/commands/crawl.py#Command:
>def run(self, args, opts):<BR>
　　if len(args) < 1:<BR>
　　　　raise UsageError()<BR>
　　elif len(args) > 1:<BR>
　　　　raise UsageError("running 'scrapy crawl' with more than one spider is no longer supported")<BR>
　　spname = args[0]<BR>
<BR>
　　self.crawler_process.crawl(spname, **opts.spargs)<BR>
　　self.crawler_process.start()<BR>

　　**这里有2个重要操作：**<BR>
　　1.调用CrawlerProcess的crawl方法，执行初始化（创建新爬虫对象）。<BR>
　　2.调用CrawlerProcess的start方法，正式运行。<BR>
CrawlerProcess便是scrapy运行过程中最根本的进程，是所有爬虫运行的基础。<BR>

In [ ]:
# --coding:utf-8--
import logging
import pprint
import signal
import warnings
 
from twisted.internet import defer
from zope.interface.exceptions import DoesNotImplement
 
try:
    # zope >= 5.0 only supports MultipleInvalid
    from zope.interface.exceptions import MultipleInvalid
except ImportError:
    MultipleInvalid = None
 
from zope.interface.verify import verifyClass
 
from scrapy import signals, Spider
from scrapy.core.engine import ExecutionEngine
from scrapy.exceptions import ScrapyDeprecationWarning
from scrapy.extension import ExtensionManager
from scrapy.interfaces import ISpiderLoader
from scrapy.settings import overridden_settings, Settings
from scrapy.signalmanager import SignalManager
from scrapy.utils.log import (
    configure_logging,
    get_scrapy_root_handler,
    install_scrapy_root_handler,
    log_scrapy_info,
    LogCounterHandler,
)
from scrapy.utils.misc import create_instance, load_object
from scrapy.utils.ossignal import install_shutdown_handlers, signal_names
from scrapy.utils.reactor import install_reactor, verify_installed_reactor
 
# scrapy 中文解释及其注释
logger = logging.getLogger(__name__)
 
#实际爬取执行的类
class Crawler:
 
    def __init__(self, spidercls, settings=None):
        if isinstance(spidercls, Spider):
            #要求传入的是类而不是实例
            raise ValueError('The spidercls argument must be a class, not an object')
 
        if isinstance(settings, dict) or settings is None:
            #转化为setting对象
            settings = Settings(settings)
 
        self.spidercls = spidercls
        self.settings = settings.copy()
        #然后使用spidercls类的update_setting方式来更新设置
        self.spidercls.update_settings(self.settings)
 
        self.signals = SignalManager(self)
        #从类的setting中的STATS_CLASS拿到stats
        self.stats = load_object(self.settings['STATS_CLASS'])(self)
        #从setting中拿到loglevel 将初始化的LogCounterHandler 加入到logging.root
        handler = LogCounterHandler(self, level=self.settings.get('LOG_LEVEL'))
        logging.root.addHandler(handler)
        # 显示出来所有被复写的setting
        d = dict(overridden_settings(self.settings))
        logger.info("Overridden settings:\n%(settings)s",
                    {'settings': pprint.pformat(d)})
 
        if get_scrapy_root_handler() is not None:
            # scrapy root handler already installed: update it with new settings
            install_scrapy_root_handler(self.settings)
        # lambda is assigned to Crawler attribute because this way it is not
        # garbage collected after leaving __init__ scope
        self.__remove_handler = lambda: logging.root.removeHandler(handler)
        #将该signals.engine_stopped信号的callback注册到self.__remove_handler函数上
        self.signals.connect(self.__remove_handler, signals.engine_stopped)
        # log格式指定
        lf_cls = load_object(self.settings['LOG_FORMATTER'])
        self.logformatter = lf_cls.from_crawler(self)
        # 扩展 还没看
        self.extensions = ExtensionManager.from_crawler(self)
 
        self.settings.freeze()
        self.crawling = False
        self.spider = None
        self.engine = None
    #defer.inlineCallbacks 装饰器 是指当使用异步调用该方法时候，
    # 该方法可以用类似同步语法的方法写异步的工作，其中yield deferred对象后
    # 后续代码会等待这个yield出去的deferred成功返回后再进行下一步
    # 其中等待时间交还给reactor。
    @defer.inlineCallbacks
    def crawl(self, *args, **kwargs):
        if self.crawling:
            raise RuntimeError("Crawling already taking place")
        self.crawling = True
 
        try:
            self.spider = self._create_spider(*args, **kwargs)
            self.engine = self._create_engine()
            #从self.spider.start_requests()中拿到requests
            start_requests = iter(self.spider.start_requests())
            #调用异步方法，开始爬虫爬取工作
            yield self.engine.open_spider(self.spider, start_requests)
            #调用核心的start方法，并将返回值包装成deferred对象
            yield defer.maybeDeferred(self.engine.start)
        except Exception:
            self.crawling = False
            if self.engine is not None:
                yield self.engine.close()
            raise
 
    def _create_spider(self, *args, **kwargs):
        #调用传入spider类的from_crawler(self, args, *kwargs)
        return self.spidercls.from_crawler(self, *args, **kwargs)
 
    def _create_engine(self):
        #ExecutionEngine(self, lambda _: self.stop()) 传入stop函数 实例化engine
        return ExecutionEngine(self, lambda _: self.stop())
 
    #重置self.crawling 为false 同时发出 异步命令self.engine.stop 包装一下self.engine.stop成为deferred对象
    @defer.inlineCallbacks
    def stop(self):
        """Starts a graceful stop of the crawler and returns a deferred that is
        fired when the crawler is stopped."""
        if self.crawling:
            self.crawling = False
            yield defer.maybeDeferred(self.engine.stop)
 
# 简单的说 如果你自己的应用用到reactor 可以考虑用这个类控制spider启停等，不然用CrawlerProcess
class CrawlerRunner:
    """
    This is a convenient helper class that keeps track of, manages and runs
    crawlers inside an already setup :mod:`~twisted.internet.reactor`.
    The CrawlerRunner object must be instantiated with a
    :class:`~scrapy.settings.Settings` object.
    This class shouldn't be needed (since Scrapy is responsible of using it
    accordingly) unless writing scripts that manually handle the crawling
    process. See :ref:`run-from-script` for an example.
    """
    #property()函数是用来指定当前属性的文件描述符类的方法，这里就是把lambda作为 他的getter 返回的是self._crawlers
    crawlers = property(
        lambda self: self._crawlers,
        doc="Set of :class:`crawlers <scrapy.crawler.Crawler>` started by "
            ":meth:`crawl` and managed by this class."
    )
    #从setting里 建立spiderloader实例
    @staticmethod
    def _get_spider_loader(settings):
        """ Get SpiderLoader instance from settings """
        cls_path = settings.get('SPIDER_LOADER_CLASS')
        #load_object(cls_path) 是将xx.xx路径转为实例的类
        loader_cls = load_object(cls_path)
        excs = (DoesNotImplement, MultipleInvalid) if MultipleInvalid else DoesNotImplement
        try:
            verifyClass(ISpiderLoader, loader_cls)
        except excs:
            warnings.warn(
                'SPIDER_LOADER_CLASS (previously named SPIDER_MANAGER_CLASS) does '
                'not fully implement scrapy.interfaces.ISpiderLoader interface. '
                'Please add all missing methods to avoid unexpected runtime errors.',
                category=ScrapyDeprecationWarning, stacklevel=2
            )
        #这里返回的是 setting中spiderloader加载目前的setting后的实例
        return loader_cls.from_settings(settings.frozencopy())
 
    def __init__(self, settings=None):
        if isinstance(settings, dict) or settings is None:
            settings = Settings(settings)
        self.settings = settings
        self.spider_loader = self._get_spider_loader(settings)
        self._crawlers = set()
        self._active = set()
        self.bootstrap_failed = False
        self._handle_twisted_reactor()
 
    @property
    def spiders(self):
        warnings.warn("CrawlerRunner.spiders attribute is renamed to "
                      "CrawlerRunner.spider_loader.",
                      category=ScrapyDeprecationWarning, stacklevel=2)
        return self.spider_loader
 
    #生成一个crawler对象 然后调用其crawl方法，其中有些管理crawler的deferred的部分 在_crawl里
    def crawl(self, crawler_or_spidercls, *args, **kwargs):
        """
        Run a crawler with the provided arguments.
        It will call the given Crawler's :meth:`~Crawler.crawl` method, while
        keeping track of it so it can be stopped later.
        If ``crawler_or_spidercls`` isn't a :class:`~scrapy.crawler.Crawler`
        instance, this method will try to create one using this parameter as
        the spider class given to it.
        Returns a deferred that is fired when the crawling is finished.
        :param crawler_or_spidercls: already created crawler, or a spider class
            or spider's name inside the project to create it
        :type crawler_or_spidercls: :class:`~scrapy.crawler.Crawler` instance,
            :class:`~scrapy.spiders.Spider` subclass or string
        :param args: arguments to initialize the spider
        :param kwargs: keyword arguments to initialize the spider
        """
        if isinstance(crawler_or_spidercls, Spider):
            raise ValueError(
                'The crawler_or_spidercls argument cannot be a spider object, '
                'it must be a spider class (or a Crawler object)')
        crawler = self.create_crawler(crawler_or_spidercls)
        return self._crawl(crawler, *args, **kwargs)
 
    def _crawl(self, crawler, *args, **kwargs):
        self.crawlers.add(crawler) #这个集合里是crawler
        d = crawler.crawl(*args, **kwargs)
        self._active.add(d) # 这个集合里是执行crawl后的deferred对象
 
        def _done(result):
            # discard 相当于不报错的remove
            self.crawlers.discard(crawler)
            self._active.discard(d)
            #a|=2等价于a=a|2(按位或)
            self.bootstrap_failed |= not getattr(crawler, 'spider', None)
            return result
        # 运行crawl前 加入到管理集合，给其deferred对象添加结束后清理管理集合的代码
        return d.addBoth(_done)
 
    def create_crawler(self, crawler_or_spidercls):
        """
        Return a :class:`~scrapy.crawler.Crawler` object.
        * If ``crawler_or_spidercls`` is a Crawler, it is returned as-is.
        * If ``crawler_or_spidercls`` is a Spider subclass, a new Crawler
          is constructed for it.
        * If ``crawler_or_spidercls`` is a string, this function finds
          a spider with this name in a Scrapy project (using spider loader),
          then creates a Crawler instance for it.
        """
        if isinstance(crawler_or_spidercls, Spider):
            raise ValueError(
                'The crawler_or_spidercls argument cannot be a spider object, '
                'it must be a spider class (or a Crawler object)')
        if isinstance(crawler_or_spidercls, Crawler):
            return crawler_or_spidercls
 
        return self._create_crawler(crawler_or_spidercls)
 
    def _create_crawler(self, spidercls):
        if isinstance(spidercls, str):
            spidercls = self.spider_loader.load(spidercls)
        # 实际实例化Crawler进行的地方 传入的是spider的类和setting
        return Crawler(spidercls, self.settings)
 
    def stop(self):
        """
        Stops simultaneously all the crawling jobs taking place.
        Returns a deferred that is fired when they all have ended.
        """
        #跟crawler不一样，这个是一个由每个crawler执行stop函数后返回的deferred对象列表
        return defer.DeferredList([c.stop() for c in list(self.crawlers)])
 
    # 跟多进程的join类似，等待所有crawler完成任务
    # 上面说到这个_active集合是所有crawler的deferred对象 把他们yield出去 以便调用后续callback
    @defer.inlineCallbacks
    def join(self):
        """
        join()
        Returns a deferred that is fired when all managed :attr:`crawlers` have
        completed their executions.
        """
        while self._active:
            yield defer.DeferredList(self._active)
    # 这个方法返回从传入setting 字符中加载 用 load_object()加载进来的recator实例 或者啥也不做
    def _handle_twisted_reactor(self):
        if self.settings.get("TWISTED_REACTOR"):
            verify_installed_reactor(self.settings["TWISTED_REACTOR"])
 
#这个crawlerProcess 是用来在不用recator的应用里使用 同时在一个进程里使用多个spider的
# 如果只是使用scrapy就不用改这个，除非想把scrapy放到你自己应用里　
# 实际上这个是上一个 CrawlerRunner 添加了reactor后的东西
class CrawlerProcess(CrawlerRunner):
    """
    A class to run multiple scrapy crawlers in a process simultaneously.
    This class extends :class:`~scrapy.crawler.CrawlerRunner` by adding support
    for starting a :mod:`~twisted.internet.reactor` and handling shutdown
    signals, like the keyboard interrupt command Ctrl-C. It also configures
    top-level logging.
    This utility should be a better fit than
    :class:`~scrapy.crawler.CrawlerRunner` if you aren't running another
    :mod:`~twisted.internet.reactor` within your application.
    The CrawlerProcess object must be instantiated with a
    :class:`~scrapy.settings.Settings` object.
    :param install_root_handler: whether to install root logging handler
        (default: True)
    This class shouldn't be needed (since Scrapy is responsible of using it
    accordingly) unless writing scripts that manually handle the crawling
    process. See :ref:`run-from-script` for an example.
    """
 
    def __init__(self, settings=None, install_root_handler=True):
        super().__init__(settings)
        #将shutdownhandler 加载为函数_signal_shutdown
        install_shutdown_handlers(self._signal_shutdown)
        configure_logging(self.settings, install_root_handler)
        log_scrapy_info(self.settings)
 
    def _signal_shutdown(self, signum, _):
        from twisted.internet import reactor
        #将shutdownhandler注册为 _signal_kill
        install_shutdown_handlers(self._signal_kill)
        signame = signal_names[signum]
        logger.info("Received %(signame)s, shutting down gracefully. Send again to force ",
                    {'signame': signame})
        #使用reactor.callFromThread(self._graceful_stop_reactor)命令调用自身的结束语句
        reactor.callFromThread(self._graceful_stop_reactor)
 
    def _signal_kill(self, signum, _):
        from twisted.internet import reactor
        #将shutdownhandler注册为signal.SIG_IGN
        install_shutdown_handlers(signal.SIG_IGN)
        signame = signal_names[signum]
        logger.info('Received %(signame)s twice, forcing unclean shutdown',
                    {'signame': signame})
        #直接将recator关闭
        reactor.callFromThread(self._stop_reactor)
 
    def start(self, stop_after_crawl=True):
        """
        This method starts a :mod:`~twisted.internet.reactor`, adjusts its pool
        size to :setting:`REACTOR_THREADPOOL_MAXSIZE`, and installs a DNS cache
        based on :setting:`DNSCACHE_ENABLED` and :setting:`DNSCACHE_SIZE`.
        If ``stop_after_crawl`` is True, the reactor will be stopped after all
        crawlers have finished, using :meth:`join`.
        :param bool stop_after_crawl: stop or not the reactor when all
            crawlers have finished
        """
        from twisted.internet import reactor
        #设置如果爬完后关闭recator的话 就添加相应的callback结束callback 这就是上面join的用处 ，如果这里不设置为true
        # 那么这个reactor就会留着不销毁
        if stop_after_crawl:
            d = self.join()
            # Don't start the reactor if the deferreds are already fired
            if d.called:
                return
            d.addBoth(self._stop_reactor)
        #加载一个配置 threadpool和dns_resolver的配置到recator
        resolver_class = load_object(self.settings["DNS_RESOLVER"])
        resolver = create_instance(resolver_class, self.settings, self, reactor=reactor)
        resolver.install_on_reactor()
        tp = reactor.getThreadPool()
        tp.adjustPoolsize(maxthreads=self.settings.getint('REACTOR_THREADPOOL_MAXSIZE'))
        reactor.addSystemEventTrigger('before', 'shutdown', self.stop)
        #启动reactor并阻塞
        reactor.run(installSignalHandlers=False)  # blocking call
 
    def _graceful_stop_reactor(self):
        # 给所有crawler 的deferred对象后面添加一个完成后销毁的动作
        d = self.stop()
        d.addBoth(self._stop_reactor)
        return d
 
    def _stop_reactor(self, _=None):
        from twisted.internet import reactor
        try:
            reactor.stop()
        except RuntimeError:  # raised if already stopped or in shutdown stage
            pass
 
    def _handle_twisted_reactor(self):
        if self.settings.get("TWISTED_REACTOR"):
            install_reactor(self.settings["TWISTED_REACTOR"], self.settings["ASYNCIO_EVENT_LOOP"])
        super()._handle_twisted_reactor()